# 🧗 Guía de Escalada Deportiva en Paraguay

Explorá las vías de escalada deportiva en Paraguay a través del mapa interactivo, una tabla filtrable y gráficos dinámicos.

---

## 🔁 Cargar shapefile, crear mapa, tabla y exportar CSV (R)


```{r cargar-datos, message=FALSE, warning=FALSE}
library(sf)
library(dplyr)
library(leaflet)
library(reactable)

# Leer shapefile
vias <- st_read("data/vias_escalada.shp", quiet = TRUE)

# Extraer coordenadas
vias <- vias %>%
  mutate(
    lon = st_coordinates(geometry)[, 1],
    lat = st_coordinates(geometry)[, 2]
  )

# Guardar como CSV para Python
vias %>%
  st_drop_geometry() %>%
  write.csv("data/vias_python.csv", row.names = FALSE)
```


🗺️ Mapa Interactivo de Vías (R)


```{r}
leaflet(vias) %>%
  addProviderTiles(providers$Esri.WorldImagery) %>%
  addCircleMarkers(
    ~lon, ~lat,
    label = ~Nombre,
    popup = ~paste(
      "<b>Vía:</b>", Nombre, "<br>",
      "<b>Grado:</b>", Grad_USA, " / ", Grad_F, "<br>",
      "<b>Altura:</b>", Altura_m, "m<br>",
      "<b>Zona:</b>", Zona, "<br>",
      "<b>Sector:</b>", Sector
    ),
    radius = 6,
    color = "#e41a1c",
    stroke = FALSE,
    fillOpacity = 0.8
  )

```

  

📊 Tabla Filtrable de Vías (R)



```{r}
reactable(
  vias %>%
    st_drop_geometry() %>%
    select(Nombre, Grad_USA, Grad_F, Altura_m, Zona, Sector),
  filterable = TRUE,
  searchable = TRUE,
  pagination = TRUE,
  defaultPageSize = 20,
  striped = TRUE,
  highlight = TRUE,
  bordered = TRUE
)

```








📈 Gráfico Interactivo por Dificultad (Python)




```{r, echo=FALSE}

library(dplyr)
library(plotly)
library(htmltools)

# Datos
vias <- st_read("data/vias_escalada.shp", quiet = TRUE) %>% 
  st_drop_geometry()

# Gráfico de barras (cantidad de vías por dificultad)
plot_ly(
  vias %>% group_by(Grad_F) %>% summarise(Vias = n()),
  x = ~Grad_F,
  y = ~Vias,
  type = "bar",
  marker = list(color = ~Vias, colorscale = "YlOrRd")
) %>%
  layout(title = "Cantidad de vías por dificultad (Grad_F)")

```